# Excercse MNIST
1. complete the training process
2. Get the MAX accuracy and MIN loss of your final model

In [25]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, utils, activations, losses, optimizers, metrics
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [26]:
tf.__version__

'2.5.0'

In [27]:
# Define Parameters
num_classes = 10
img_size = 28

#### Prepare Data

In [28]:
# Load Data from file
with np.load('mnist.npz', allow_pickle=True) as f:
    
    x_train, y_train = f['x_train'], f['y_train']
    x_test = f['x_test']

# Data shape
x_train.shape, y_train.shape, x_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28))

In [29]:
# split dataset
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

x_train.shape, x_val.shape, x_train.shape, x_val.shape

((48000, 28, 28), (12000, 28, 28), (48000, 28, 28), (12000, 28, 28))

In [30]:
# Flatten img to a vector
x_train = x_train.reshape(x_train.shape[0], -1)
x_val = x_val.reshape(x_val.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# noramlize
x_train = x_train/255
x_test = x_test/255

# one hot encoding
y_train = utils.to_categorical(y_train, num_classes=num_classes)
y_val = utils.to_categorical(y_val, num_classes=num_classes)

x_train.shape, x_val.shape, y_train.shape, y_val.shape

((48000, 784), (12000, 784), (48000, 10), (12000, 10))

#### Build Model

In [31]:
# Build your own model
model = models.Sequential([
               layers.Dense(units=64,activation='elu',input_shape=(img_size*img_size,)),
               layers.Dense(units=64,activation='elu'),
               layers.Dense(units=128,activation='elu'),
               layers.Dense(num_classes,activation='softmax')
])

In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 64,010
Trainable params: 64,010
Non-trainable params: 0
_________________________________________________________________


#### Training

In [34]:
model.compile(optimizer=optimizers.Adam(),
       loss=losses.categorical_crossentropy,
       metrics=['accuracy'])

logs = model.fit(x_train,y_train,
          batch_size=500,
          epochs=25,
          verbose=1,
          validation_data=(x_val,y_val))

Epoch 1/25
96/96 [==============================] - 1s 8ms/step - loss: 0.0680 - accuracy: 0.9789 - val_loss: 56.6742 - val_accuracy: 0.7867
Epoch 2/25
96/96 [==============================] - 1s 10ms/step - loss: 0.0546 - accuracy: 0.9836 - val_loss: 49.7996 - val_accuracy: 0.8074
Epoch 3/25
96/96 [==============================] - 1s 6ms/step - loss: 0.0481 - accuracy: 0.9852 - val_loss: 60.7826 - val_accuracy: 0.7955
Epoch 4/25
96/96 [==============================] - 1s 6ms/step - loss: 0.0431 - accuracy: 0.9873 - val_loss: 70.5227 - val_accuracy: 0.7840
Epoch 5/25
96/96 [==============================] - 1s 10ms/step - loss: 0.0364 - accuracy: 0.9891 - val_loss: 87.2085 - val_accuracy: 0.7513
Epoch 6/25
96/96 [==============================] - 1s 7ms/step - loss: 0.0320 - accuracy: 0.9903 - val_loss: 87.8939 - val_accuracy: 0.7622
Epoch 7/25
96/96 [==============================] - 1s 6ms/step - loss: 0.0282 - accuracy: 0.9916 - val_loss: 100.8044 - val_accuracy: 0.7497
Epoch 8/25

#### Generate file for Kaggle

In [35]:
# predict by your model
predictions = model.predict(x_test)
predictions = np.argmax(predictions, axis=-1)

df = pd.DataFrame()
df['Id'] = [str(i) for i in range(len(x_test))]
df['Category'] = predictions
df.to_csv('result.csv', index=None)
df

,Id,Category
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4
...,...,...
9995,9995,2
9996,9996,3
9997,9997,4
9998,9998,5
